# Dataset
We will explore this dataset: https://archive.ics.uci.edu/ml/datasets/EEG+Eye+State#

> All data is from one continuous EEG measurement with the Emotiv EEG Neuroheadset. The duration of the measurement was 117 seconds. The eye state was detected via a camera during the EEG measurement and added later manually to the file after analysing the video frames. '1' indicates the eye-closed and '0' the eye-open state. All values are in chronological order with the first measured value at the top of the data.

In [2]:
import tensorflow as tf
data_dir = "../../data/raw"
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00264/EEG%20Eye%20State.arff"
datapath = tf.keras.utils.get_file(
        "eeg", origin=url, untar=False, cache_dir=data_dir
    )

2022-06-06 07:13:30.913435: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-06 07:13:30.913473: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


You can load the arff file with scipy

In [3]:
from scipy.io import arff
data = arff.loadarff(datapath)

In [4]:
datapath

'../../data/raw/datasets/eeg'

The data is a tuple of a description and observations

In [5]:
len(data), type(data)

(2, tuple)

Description

In [6]:
data[1]

Dataset: EEG_DATA
	AF3's type is numeric
	F7's type is numeric
	F3's type is numeric
	FC5's type is numeric
	T7's type is numeric
	P7's type is numeric
	O1's type is numeric
	O2's type is numeric
	P8's type is numeric
	T8's type is numeric
	FC6's type is numeric
	F4's type is numeric
	F8's type is numeric
	AF4's type is numeric
	eyeDetection's type is nominal, range is ('0', '1')

There are about 15k observations

In [7]:
line = data[0][0]
observation = []
obs = []
for index, i in enumerate(line):
    if index != 14:
        observation.append(i)
observation = torch.Tensor(observation)
observation

torch.stack(observation)

NameError: name 'torch' is not defined

The observations are tuples of floats and a byte as label

In [91]:
first_label = int(data[0][0][14])
label = first_label
chunck = []
chuncks = []
for line in data[0]:
    if int(line[14]) == label:
        observation = []
        for index, i in enumerate(line):
            if index != 14:
                observation.append(i)
        observation = torch.Tensor(observation)
        chunck.append(observation)
    else:
        chunck_tuple = (label, torch.stack(chunck))
        chuncks.append(chunck_tuple)
        label = int(line[14])
        chunck = []
        observation = []
        for index, i in enumerate(line):
            if index != 14:
                observation.append(i)
        observation = torch.Tensor(observation)
chunck_tuple = (label, torch.stack(chunck))
chuncks.append(chunck_tuple)


def window(x: Tensor, n_time: int) -> Tensor:
    """
    Generates and index that can be used to window a timeseries.
    E.g. the single series [0, 1, 2, 3, 4, 5] can be windowed into 4 timeseries with
    length 3 like this:

    [0, 1, 2]
    [1, 2, 3]
    [2, 3, 4]
    [3, 4, 5]

    We now can feed 4 different timeseries into the model, instead of 1, all
    with the same length.
    """
    n_window = len(x) - n_time + 1
    time = torch.arange(0, n_time).reshape(1, -1)
    window = torch.arange(0, n_window).reshape(-1, 1)
    idx = time + window
    return idx

item = chuncks
for item in chuncks:
    print(len(item[1]), item[0])

print(item[1].shape)
y = item[0]


188 0
682 1
464 0
301 1
537 0
456 1
266 0
26 1
414 0
1009 1
891 0
683 1
724 0
2400 1
2050 0
970 1
651 0
42 1
204 0
51 1
1188 0
71 1
669 0
20 1
torch.Size([20, 14])


In [121]:
idx = window(item[1], n_time = 50)
idx

RuntimeError: upper bound and larger bound inconsistent with step sign

In [103]:
def window(x: Tensor, n_time: int) -> Tensor:
    """
    Generates and index that can be used to window a timeseries.
    E.g. the single series [0, 1, 2, 3, 4, 5] can be windowed into 4 timeseries with
    length 3 like this:

    [0, 1, 2]
    [1, 2, 3]
    [2, 3, 4]
    [3, 4, 5]

    We now can feed 4 different timeseries into the model, instead of 1, all
    with the same length.
    """
    n_window = len(x) - n_time + 1
    time = torch.arange(0, n_time).reshape(1, -1)
    window = torch.arange(0, n_window).reshape(-1, 1)
    idx = time + window
    return idx

item[1].shape
len(window(item[1], 19))

2

In [ ]:
window_size = 30
count = 0
for i in self.dataset:
    lenght_observation = i[1].shape[0]
    items_obs = lenght_observation-window_size+1
    count += items_obs
return count

b'0'

In [ ]:
import numpy as np
np.array(labels).mean()

0.4487983978638184

About 45% of the data has closed eyes.

# Excercises 1

- create a get_eeg function that downloads the data to a given path
- build a Dataset that yields a ($X, y$) tuple of tensors. $X$ should be sequential in time. Remember: a dataset should implement `__get_item__` and `__len__`.
- note that you could model this as both a classification task, but also as a sequence-to-sequence task! For this excercise, make it a classification task with consecutive 0s or 1s only.
- Note that, for a training task, a seq2seq model will probably be more realistic. However, the classification is a nice excercise because it is harder to set up.
- figure out what the length distribution is of your dataset: how many timestamps do you have for every consecutive sequence of 0s and 1s? On average, median, min, max?
- create a dataloader that yields timeseries with (batch, sequence_lenght). You can implement: windowed, padded and batched.
    1. yielding a windowed item should be the easy level
    2. yielding windowed and padded is medium level 
    3. yielding windowed, padded and batched is expert level, because the windowing will cause the timeseries to have different sizes. You will need to buffer before you can yield a batch.

1. Upload this to github. 
2. Put your dev notebooks in a seperate folder
3. Put all your functions in the src folder
4. Use a formater & linter
5. Add a single notebook, that sources the src folder. Indicate which level you got (1, 2 or 3)
6. and that shows your dataloader works:
    - it should not give errors because it runs out of data! Either let is stop by itself, or run forever.
    - batchsize should be consistent (in case 1 and 2, batchsize is 1)
    - sequence length is allowed to vary

The first excercise is ex1, this one is ex2. You will get $max(ex1, average(ex1, ex2))$ as a final remark.
Level 3 can get you an 11, because it exceeds expectation.

# Excercise 2
- build a Dataset that yields sequences of X, y. This time, y is a sequence and can contain both 0s and 1s
- create a Dataloader with this
- Test appropriate architectures (RNN, Attention)
- for the loss, note that you will need a BCELoss instead of a CrossEntroyLoss

In [ ]:
import random


list_index

[1,
 3,
 6,
 4,
 7,
 22,
 20,
 21,
 10,
 16,
 2,
 5,
 9,
 15,
 8,
 0,
 18,
 11,
 17,
 19,
 14,
 12,
 13]

In [149]:
from __future__ import annotations
import random
import shutil
from datetime import datetime
from pathlib import Path
from typing import Callable, Dict, Iterator, List, Optional, Sequence, Tuple, Union
import numpy as np
import tensorflow as tf
import torch
from loguru import logger
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from scipy.io import arff

Tensor = torch.Tensor

def get_eeg(data_dir: Path = "../../data/raw") -> Path:
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00264/EEG%20Eye%20State.arff"
    datapath = tf.keras.utils.get_file(
        "eeg", origin=url, untar=False, cache_dir=data_dir
    )
    datapath = Path(datapath)
    logger.info(f"Data is downloaded to {datapath}.")
    return datapath

class BaseDataset:
    def __init__(self, datapath: Path):
        self.path = datapath
        self.data = self.process_data()

        
    def process_data(self) -> None:
        data = arff.loadarff(self.path)
        first_label = int(data[0][0][14])
        label = first_label
        chunck = []
        chuncks = []
        for line in data[0]:
            if int(line[14]) == label:
                observation = []
                for index, i in enumerate(line):
                    if index != 14:
                        observation.append(i)
                observation = torch.Tensor(observation)
                chunck.append(observation)
            else:
                chunck_tuple = (label, torch.stack(chunck))
                chuncks.append(chunck_tuple)
                label = int(line[14])
                chunck = []
                observation = []
                for index, i in enumerate(line):
                    if index != 14:
                        observation.append(i)
                observation = torch.Tensor(observation)
        chunck_tuple = (label, torch.stack(chunck))
        chuncks.append(chunck_tuple)
        return chuncks

    def __getitem__(self, idx):
        item = self.data[idx]
        x = item[1]
        y = item[0]
        return x,y


    def __len__(self):
        length = len(self.data)
        return length

dataloader = BaseDataset(datapath = get_eeg())
dataloader.__getitem__(1)

2022-06-06 09:26:34.176 | INFO     | __main__:get_eeg:26 - Data is downloaded to ../../data/raw/datasets/eeg.


(tensor([[4438.4600, 4005.6399, 4287.6899,  ..., 4356.4102, 4811.2798,
          4546.6699],
         [4432.3101, 3998.4600, 4281.5400,  ..., 4351.2798, 4795.8999,
          4533.8501],
         [4427.1802, 3990.2600, 4281.0298,  ..., 4343.0801, 4783.5898,
          4530.7700],
         ...,
         [4246.6699, 4002.0500, 4247.1802,  ..., 4263.5898, 4525.6401,
          4283.0801],
         [4249.2300, 4005.6399, 4245.1299,  ..., 4265.1299, 4533.3301,
          4288.2100],
         [4253.3301, 4015.8999, 4252.8198,  ..., 4267.1802, 4543.5898,
          4301.0298]]),
 1)

In [270]:
listpossibilities = [(0, 168), (1, 662), (2, 444), (3, 281), (4, 517), (5, 436), (6, 246), (7, 6), (8, 394), (9, 989), (10, 871), (11, 663), (12, 704), (13, 2380), (14, 2030), (15, 950), (16, 631), (17, 22), (18, 184), (19, 31), (20, 1168), (21, 51), (22, 649), (23, 0)]
listpossibilities

count = 0
window_size =10
dict_iterator = {}

for i in listpossibilities:
    windowedItems = i[1] -window_size + 1
    count += windowedItems
        


for item in range(count):
    obs = random.randint(0,23)
    dict_iterator[item] = (obs, random.randint(0, listpossibilities[obs][1]))

dict_iterator

{0: (20, 487),
 1: (11, 432),
 2: (13, 783),
 3: (16, 20),
 4: (22, 409),
 5: (17, 3),
 6: (23, 0),
 7: (5, 49),
 8: (21, 8),
 9: (12, 326),
 10: (6, 28),
 11: (16, 535),
 12: (11, 541),
 13: (19, 27),
 14: (11, 260),
 15: (19, 12),
 16: (20, 237),
 17: (6, 4),
 18: (14, 1760),
 19: (12, 61),
 20: (17, 7),
 21: (15, 412),
 22: (5, 19),
 23: (7, 1),
 24: (8, 294),
 25: (12, 354),
 26: (16, 567),
 27: (7, 4),
 28: (4, 368),
 29: (11, 204),
 30: (19, 5),
 31: (7, 4),
 32: (6, 53),
 33: (19, 7),
 34: (3, 94),
 35: (7, 0),
 36: (2, 431),
 37: (7, 6),
 38: (18, 9),
 39: (22, 41),
 40: (20, 719),
 41: (1, 59),
 42: (22, 78),
 43: (20, 303),
 44: (8, 61),
 45: (15, 159),
 46: (11, 392),
 47: (16, 425),
 48: (16, 401),
 49: (0, 65),
 50: (6, 218),
 51: (10, 150),
 52: (12, 212),
 53: (18, 171),
 54: (22, 442),
 55: (12, 632),
 56: (12, 106),
 57: (10, 288),
 58: (7, 4),
 59: (4, 426),
 60: (8, 278),
 61: (22, 90),
 62: (18, 19),
 63: (0, 167),
 64: (17, 11),
 65: (13, 839),
 66: (14, 1776),
 67

In [301]:
class BaseDataIterator:
    def __init__(self, dataset: BaseDataset, window_size: int, batchsize: int) -> None:
        self.dataset = dataset #set dataset
        self.batchsize = batchsize #set batchsize
        self.window_size = window_size #set windowsize

    def __len__(self) -> int:
        window_size = self.window_size
        count = 0 #init count
        for i in self.dataset: #loop through all observations
            lenght_observation = i[0].shape[0] #get nr of rows in the observation
            items_obs = lenght_observation-window_size+1 #get possible nr of windows from observation
            count += items_obs #increment count
        return count #return max nr of items

    def __iter__(self) -> BaseDataIterator: 
        length = len(self) #get the max nr of iterations
        self.dict_iterator = {} #init dict to store indexes 
        lengthobs = len(self.dataset) #get nr of observations
        index_tuples = [] #to store max nr tuples in
        for i in range(lengthobs):
            x,y = dataset.__getitem__(i) 
            lenght_item = x.shape[0]-self.window_size+1 
            index_tuples.append((i, lenght_item)) 
        for item in range(count):
            obs = random.randint(0,23)
            self.dict_iterator[item] = (obs, random.randint(0, index_tuples[obs][1]))
        self.index = 0
        return self

    def __next__(self) -> Tuple[Tensor, Tensor]:
        X, Y = self.batchloop()  # noqa N806
        return torch.tensor(X), torch.tensor(Y)

    def window(self, x: Tensor, n_time: int) -> Tensor: #function to get windowed items from observation
        """
        Generates and index that can be used to window a timeseries.
        E.g. the single series [0, 1, 2, 3, 4, 5] can be windowed into 4 timeseries with
        length 3 like this:

        [0, 1, 2]
        [1, 2, 3]
        [2, 3, 4]
        [3, 4, 5]

        We now can feed 4 different timeseries into the model, instead of 1, all
        with the same length.
        """
        n_window = len(x) - n_time + 1
        time = torch.arange(0, n_time).reshape(1, -1)
        window = torch.arange(0, n_window).reshape(-1, 1)
        idx = time + window
        return idx

    def get_windowed_item(self):
        print(self.index)
        window_size = self.window_size #get window size
        itemIndexes = self.dict_iterator[self.index]
        print(itemIndexes)
        x,y = self.dataset.__getitem__(itemIndexes[0]) #get the item and store in x and y
        if self.window_size > int(x.shape[0]): #if the window size is larger than the nr of lines in observation
            idx = self.window(x, x.shape[0]) #take the whole item
        else: # if the window size is smaller than the nr of lines in the observation
            idx = self.window(x, window_size) #use window function to get windowed sample
        currentObservation = x[idx] #apply idx to currentobservation
        currentitem = currentObservation[itemIndexes[1]] #get one windowed item
        return currentitem, y #return windowed item and its class

    def batchloop(self) -> Tuple[List, List]:
        X = []  # noqa N806
        Y = []  # noqa N806
        for _ in range(self.batchsize): 
            x,y = self.get_windowed_item()
            X.append(x)
            Y.append(y)
            self.index += 1
        return X, Y

class PaddedDatagenerator(BaseDataIterator):
    """Iterator with additional padding of X

    Args:
        BaseDataIterator (_type_): _description_
    """

    def __init__(self, dataset: BaseDataset, window_size: int, batchsize: int) -> None:
        super().__init__(dataset, window_size, batchsize)

    def __next__(self):
        if self.index <= (len(self) - self.batchsize):
            X, Y = self.batchloop()
            X_ = pad_sequence(X, batch_first=True, padding_value=0) #if there are shorter sequences, add padding
            return X_, torch.tensor(Y)
        else:
            raise StopIteration

dataset = BaseDataset(datapath = get_eeg())
loader = PaddedDatagenerator(dataset = dataset, window_size=30, batchsize=32)

#moeten we ervoor zorgen dat elk window max 1 keer gebruikt wordt of is het ok dat er een kleine 
# kans bestaat dat iets dubben gebruikt wordt

#omgaan met train en test set

2022-06-06 11:17:11.622 | INFO     | __main__:get_eeg:26 - Data is downloaded to ../../data/raw/datasets/eeg.


In [302]:
iterator = iter(loader)
batch = next(loader)
batch = next(loader)
batch = next(loader)

(7, 1)


IndexError: index 1 is out of bounds for dimension 0 with size 1

In [243]:
batch[0].shape

torch.Size([32, 21, 14])

In [244]:
batch[1]

tensor([1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
        0, 1, 1, 1, 0, 0, 1, 0])

In [234]:
i = iter(loader)
next(i)

(tensor([[[4285.6401, 3993.3301, 4265.1299,  ..., 4255.3799, 4589.2300,
           4352.3101],
          [4292.3101, 3998.9700, 4262.5601,  ..., 4258.4600, 4597.9502,
           4352.8198],
          [4291.2798, 4003.0801, 4264.1001,  ..., 4265.1299, 4601.5400,
           4347.6899],
          ...,
          [4314.8701, 4018.9700, 4280.5098,  ..., 4267.6899, 4594.8701,
           4368.7202],
          [4308.7202, 4017.9500, 4275.8999,  ..., 4265.1299, 4590.2598,
           4368.7202],
          [4303.5898, 4015.8999, 4272.3101,  ..., 4263.5898, 4581.0298,
           4366.6699]],
 
         [[4363.5898, 4046.1499, 4288.7202,  ..., 4293.8501, 4618.9702,
           4401.0298],
          [4351.7900, 4039.4900, 4279.4902,  ..., 4293.8501, 4612.8198,
           4394.3599],
          [4342.5601, 4032.8201, 4278.4600,  ..., 4289.7402, 4609.2300,
           4392.3101],
          ...,
          [4298.4600, 3999.4900, 4267.6899,  ..., 4278.9702, 4595.8999,
           4355.3799],
          [4294.3

In [ ]:

class PaddedDatagenerator(BaseDataIterator):
    """Iterator with additional padding of X

    Args:
        BaseDataIterator (_type_): _description_
    """

    def __init__(self, dataset: BaseDataset, batchsize: int) -> None:
        super().__init__(dataset, batchsize)

    def __next__(self) -> Tuple[Tensor, Tensor]:
        if self.index <= (len(self.dataset) - self.batchsize):
            X, Y = self.batchloop()  # noqa N806
            X_ = pad_sequence(X, batch_first=True, padding_value=0)  # noqa N806
            return X_, torch.tensor(Y)
        else:
            raise StopIteration


class BaseDatastreamer:
    """This datastreamer will never stop
    The dataset should have a:
        __len__ method
        __getitem__ method

    """

    def __init__(
        self,
        dataset: BaseDataset,
        batchsize: int,
        preprocessor: Optional[Callable] = None,
    ) -> None:
        self.dataset = dataset
        self.batchsize = batchsize
        self.preprocessor = preprocessor
        self.size = len(self.dataset)
        self.reset_index()

    def __len__(self) -> int:
        return int(len(self.dataset) / self.batchsize)

    def reset_index(self) -> None:
        self.index_list = np.random.permutation(self.size)
        self.index = 0

    def batchloop(self) -> Sequence[Tuple]:
        batch = []
        for _ in range(self.batchsize):
            x, y = self.dataset[int(self.index_list[self.index])]
            batch.append((x, y))
            self.index += 1
        return batch

    def stream(self) -> Iterator:
        while True:
            if self.index > (self.size - self.batchsize):
                self.reset_index()
            batch = self.batchloop()
            if self.preprocessor is not None:
                X, Y = self.preprocessor(batch)  # noqa N806
            else:
                X, Y = zip(*batch)  # noqa N806
            yield X, Y


In [ ]:
from __future__ import annotations
import random
import shutil
from datetime import datetime
from pathlib import Path
from typing import Callable, Dict, Iterator, List, Optional, Sequence, Tuple, Union
import numpy as np
import tensorflow as tf
import torch
from loguru import logger
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from src.data import data_tools
from src.data.data_tools import PaddedDatagenerator, TSDataset

def window(x: Tensor, n_time: int) -> Tensor:
    """
    Generates and index that can be used to window a timeseries.
    E.g. the single series [0, 1, 2, 3, 4, 5] can be windowed into 4 timeseries with
    length 3 like this:

    [0, 1, 2]
    [1, 2, 3]
    [2, 3, 4]
    [3, 4, 5]

    We now can feed 4 different timeseries into the model, instead of 1, all
    with the same length.
    """
    n_window = len(x) - n_time + 1
    time = torch.arange(0, n_time).reshape(1, -1)
    window = torch.arange(0, n_window).reshape(-1, 1)
    idx = time + window
    return idx



def get_eeg(data_dir: Path = "../../data/raw") -> Path:
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00264/EEG%20Eye%20State.arff"
    datapath = tf.keras.utils.get_file(
        "eeg", origin=url, untar=False, cache_dir=data_dir
    )
    datapath = Path(datapath)
    logger.info(f"Data is downloaded to {datapath}.")
    return datapath

class Dataloader:
    def __init__(self, datapath: Path):
        self.path = datapath
        self.dataset = []
        self.process_data()

    def __get_item__():
        pass

    def __len__():
        pass

    def process_data(self) -> None:
        file = self.path
        x_ = np.genfromtxt(file)[:, 3:]
        x = torch.tensor(x_).type(torch.float32)
        y = torch.tensor(int(file.parent.name) - 1)
        self.dataset.append((x, y))

class BaseDataset:
    def __init__(self, paths: List[Path]) -> None:
        self.paths = paths
        random.shuffle(self.paths)
        self.dataset = []
        self.process_data()

    def process_data(self) -> None:
        for file in tqdm(self.paths):
            x_ = np.genfromtxt(file)[:, 3:]
            x = torch.tensor(x_).type(torch.float32)
            y = torch.tensor(int(file.parent.name) - 1)
            self.dataset.append((x, y))

    def __len__(self) -> int:
        return len(self.dataset)

    def __getitem__(self, idx: int) -> Tuple[Tensor, int]:
        return self.dataset[idx]

class BaseDataIterator:
    def __init__(self, dataset: BaseDataset, batchsize: int):
        self.dataset = dataset
        self.batchsize = batchsize

    def __len__(self) -> int:
        # the lenght is the amount of batches
        return int(len(self.dataset) / self.batchsize)

    def __iter__(self) -> BaseDataIterator:
        # initialize index
        self.index = 0
        self.index_list = torch.randperm(len(self.dataset))
        return self
    
    def batchloop(self) -> Tuple[Tensor, Tensor]:
        X = []  # noqa N806
        Y = []  # noqa N806
        # fill the batch
        for _ in range(self.batchsize):
            x, y = self.dataset[int(self.index_list[self.index])]
            X.append(x)
            Y.append(y)
            self.index += 1
        return X, Y

    def __next__(self) -> Tuple[Tensor, Tensor]:
        if self.index <= (len(self.dataset) - self.batchsize):
            X, Y = self.batchloop()
            return X, Y
        else:
            raise StopIteration

# BCELoss example
In this example, which input would you prefer for the given target?

In [25]:
import torch
import torch.nn as nn

input1 = torch.tensor([0.1, 0.1, 0.7, 0.9])
input2 = torch.tensor([0.1, 0.3, 0.6, 0.7])
target = torch.tensor([0., 0., 1., 1.])

So, which loss should you pick? CrossEntropyLoss won't work:

In [32]:
loss = nn.CrossEntropyLoss()
try:
    loss(input1, target)
except Exception as e:
    print(e)

Dimension out of range (expected to be in range of [-1, 0], but got 1)


You will need BCELoss for this.
Binary cross entropy loss works like this:
$$X = {x_i, \dots, x_n}$$

$$l_i =-(y_i \cdot log(x_i) + (1-y_i) \cdot log(1-x_i))$$
$$BCELoss = mean(l)$$

Note that the labels are assumed to be either 0 or 1 (hence, the binary part).
If a label is 0, only the second part is relevent. If the label is 1, only the first part is relevant. the default reduction is "mean":

$$
BCEloss = 
\begin{cases}
mean(-log(1 - x_i)) & \text{if\,} y = 0\\
mean(-log(x_i)) & \text{if\,} y = 1
\end{cases}
$$



 We can see this works nice for a sequence of 0s and 1s.
You can see that input1 is preferred, because it is more certain of the cases.

In [34]:
loss = nn.BCELoss()
loss(input1, target), loss(input2, target)

(tensor(0.1682), tensor(0.3324))

Or a more generic example

In [35]:
m = nn.Sigmoid() # make sure outputs are between 0 and 1
X = torch.randn(100) # generate 100 random inputs
yhat = m(X) # our dummy model

p = torch.ones_like(yhat) / 2
y = torch.bernoulli(p) # we create a random label sequence of 0s and 1s
loss(yhat, y)

tensor(0.8594)